# PaaS AI - Getting Started Guide

This notebook walks you through setting up and using PaaS AI from scratch. We'll cover:

1. **Environment Setup** - Installing dependencies and setting up the environment
2. **Custom Profile Creation** - Creating and activating a custom configuration profile
3. **Documentation Server** - Running the example PaaS documentation with npm
4. **Interactive Chat** - Starting the AI chat interface

## Prerequisites

- Python 3.11 or 3.12
- Poetry (for dependency management)
- Node.js 18+ (for documentation server)
- OpenAI API key (for AI functionality)

Let's get started! 🚀


## Step 1: Environment Setup

First, let's check our current directory and install the project dependencies using Poetry.


In [ ]:
import os
import subprocess
from pathlib import Path

# Check current directory
current_dir = Path.cwd()
print(f"Current directory: {current_dir}")

# Verify we're in the paas-ai project root
if not (current_dir / "pyproject.toml").exists():
    print("❌ Not in paas-ai project root. Please navigate to the project directory.")
else:
    print("✅ Found pyproject.toml - we're in the right place!")


In [ ]:
# Install project dependencies using Make
print("📦 Installing all dependencies...")
result = subprocess.run(["make", "install-all"], capture_output=False, text=True)

if result.returncode == 0:
    print("✅ All dependencies installed successfully!")
else:
    print(f"❌ Installation failed: {result.stderr}")
    print("Please run 'make install-all' manually")


## Step 2: Environment Variables Setup

Let's set up the required environment variables. You'll need an OpenAI API key for the AI functionality.


In [ ]:
# Check if .env file exists, if not create from example
env_file = Path(".env")
env_example = Path("env.example")

if not env_file.exists() and env_example.exists():
    # Copy env.example to .env
    with open(env_example, 'r') as src, open(env_file, 'w') as dst:
        dst.write(src.read())
    print("✅ Created .env file from env.example")
elif env_file.exists():
    print("✅ .env file already exists")
else:
    print("❌ No env.example found. Creating basic .env file...")
    with open(env_file, 'w') as f:
        f.write("OPENAI_API_KEY=your_openai_api_key_here\n")
    print("✅ Created basic .env file")

print("\n⚠️  Please edit the .env file and add your OpenAI API key:")
print("   OPENAI_API_KEY=your_actual_api_key_here")


## Step 3: Create and Activate Custom Profile

Now let's create a custom configuration profile. This allows us to customize the AI behavior, embedding models, and other settings.


In [ ]:
# Check current config.yaml
config_file = Path.home() / ".paas-ai" / "config.yaml"

if config_file.exists():
    print("✅ Found existing config.yaml")
    
    # Read and display current profile
    with open(config_file, 'r') as f:
        content = f.read()
        
    # Extract current profile
    import yaml
    config = yaml.safe_load(content)
    current_profile = config.get('current', 'default')
    
    print(f"📋 Current active profile: {current_profile}")
    print(f"📋 Available profiles: {list(config.get('profiles', {}).keys())}")
else:
    print("❌ No config.yaml found. Let's create one...")
    
    # Initialize config
    result = subprocess.run(["poetry", "run", "paas-ai", "config", "init"], 
                           capture_output=True, text=True)
    
    if result.returncode == 0:
        print("✅ Config initialized successfully!")
    else:
        print(f"❌ Config initialization failed: {result.stderr}")


In [ ]:
# Create a new custom profile called "notebook_demo"
profile_name = "notebook_demo"

print(f"🔧 Creating custom profile: {profile_name}")

# Create profile using CLI (correct command is "add-profile")
result = subprocess.run([
    "poetry", "run", "paas-ai", "config", "add-profile", profile_name,
    "--embedding-type", "sentence_transformers",
    "--embedding-model", "all-MiniLM-L6-v2",
    "--vectorstore-type", "chroma",
    "--activate"
], capture_output=True, text=True)

if result.returncode == 0:
    print(f"✅ Profile '{profile_name}' created successfully!")
    print(result.stdout)
else:
    print(f"❌ Profile creation failed: {result.stderr}")
    print("You can create it manually or use an existing profile.")


In [ ]:
# Activate the custom profile (if not already activated by --activate flag)
print(f"🔄 Setting profile as current: {profile_name}")

result = subprocess.run([
    "poetry", "run", "paas-ai", "config", "set-current", profile_name
], capture_output=True, text=True)

if result.returncode == 0:
    print(f"✅ Profile '{profile_name}' set as current successfully!")
    print(result.stdout)
else:
    print(f"❌ Profile activation failed: {result.stderr}")
    print("The profile may already be active from the --activate flag above")


## Step 4: Run Example PaaS Documentation

Now let's start the documentation server. This provides example PaaS configurations and documentation that the AI can reference.


In [ ]:
# Navigate to the docs directory and install dependencies
docs_dir = Path("examples/paas/docs")
print("📦 Setting up documentation server...")


In [ ]:
# Install npm dependencies and start server
original_dir = os.getcwd()
os.chdir(docs_dir)

# Install dependencies
subprocess.run(["npm", "install"], capture_output=True, text=True)
print("✅ Dependencies installed")

# Return to project root
os.chdir(original_dir)


In [ ]:
# Start the documentation server in background
import threading
import time

def start_docs_server():
    os.chdir(docs_dir)
    subprocess.run(["npm", "start"], capture_output=False)

server_thread = threading.Thread(target=start_docs_server, daemon=True)
server_thread.start()
time.sleep(3)

print("🚀 Documentation server started at http://localhost:3000")
os.chdir(original_dir)


## Step 4.5: Clean Up Previous Data (Optional)

Before loading new knowledge, you can optionally clean up any existing RAG data or conversation history from previous runs. This ensures a fresh start and prevents conflicts.


In [ ]:
# Optional: Clean up existing data for a fresh start
import shutil

# Define paths we'll need
profile_rag_dir = Path(f"rag_data/{profile_name}")

cleanup_choice = input("Would you like to clean up existing RAG data and conversation history? (y/N): ")

if cleanup_choice.lower() in ['y', 'yes']:
    print("🧹 Cleaning up existing data...")
    
    # Clean up RAG data directory
    rag_data_dir = Path("rag_data")
    if rag_data_dir.exists():
        try:
            shutil.rmtree(rag_data_dir)
            print("✅ Removed existing RAG data directory")
        except Exception as e:
            print(f"⚠️  Could not remove RAG data directory: {e}")
    else:
        print("📝 No existing RAG data directory found")
    
    # Clean up conversation history (only the conversations.db file, NOT the config!)
    conversations_db = Path.home() / ".paas-ai" / "conversations.db"
    if conversations_db.exists():
        try:
            conversations_db.unlink()
            print("✅ Removed existing conversation history")
        except Exception as e:
            print(f"⚠️  Could not remove conversation history: {e}")
    else:
        print("📝 No existing conversation history found")
    
    print("✅ Cleanup completed! Starting with fresh data.")
    
else:
    print("⏭️  Skipping cleanup - keeping existing data")
    print("💡 If you encounter issues, you can run cleanup manually later")


## Step 5: Load Knowledge Base (Optional)

Before starting the chat, let's optionally load some knowledge into the RAG system. This step is optional but recommended for better AI responses.


In [ ]:
# Check if we have any existing knowledge base (profile-specific path)
profile_rag_dir = Path(f"rag_data/{profile_name}")

if profile_rag_dir.exists():
    print(f"✅ Found profile-specific RAG data: {profile_rag_dir}")
    
    # Check for ChromaDB files
    chroma_db = profile_rag_dir / "chroma.sqlite3"
    if chroma_db.exists():
        print("✅ Knowledge base is set up!")
    else:
        print("📝 RAG directory exists but no data found")
else:
    print("📝 No knowledge base found for this profile")
    print("💡 We'll load the data in the next step")


In [ ]:
# Load PaaS documentation using the CSV file
load_knowledge = input("Would you like to load the PaaS documentation into the knowledge base? (y/N): ")

if load_knowledge.lower() in ['y', 'yes']:
    print("📚 Loading PaaS documentation from CSV...")
    print("⏳ This may take a few minutes...")
    
    # Load documentation using the CSV file (with real-time output)
    print("🔄 Running command with live output:")
    print("poetry run paas-ai rag resources add-batch --csv-file examples/paas/paas_docs_urls.csv")
    print("-" * 80)
    
    result = subprocess.run([
        "poetry", "run", "paas-ai", "rag", "resources", "add-batch",
        "--csv-file", "examples/paas/paas_docs_urls.csv"
    ], text=True)  # Remove capture_output to see live output
    
    print("-" * 80)
    
    if result.returncode == 0:
        print("✅ Documentation loaded successfully!")
        # Check if data was actually loaded
        if profile_rag_dir.exists() and (profile_rag_dir / "chroma.sqlite3").exists():
            print(f"✅ Verified: Knowledge base created at {profile_rag_dir}")
        else:
            print("⚠️  Warning: Command succeeded but no data files found")
    else:
        print(f"❌ Documentation loading failed with return code: {result.returncode}")
        print("💡 You can try loading it manually later")
else:
    print("⏭️  Skipping knowledge base loading")
    print("💡 You can load it later using: poetry run paas-ai rag resources add-batch --csv-file examples/paas/paas_docs_urls.csv")


## Step 6: Start Interactive Chat

Now for the exciting part - let's start the interactive chat with the AI! This will launch the PaaS AI chat interface where you can ask questions about PaaS configurations, get help with deployments, and more.


In [ ]:
# Ready to start chat!
print("🤖 Everything is set up and ready to go!")


In [ ]:
# Display chat startup information
print("🤖 Ready to start PaaS AI Chat!")
print("\n" + "="*60)
print("🚀 PAAS AI INTERACTIVE CHAT")
print("="*60)
print("\n💡 What you can do:")
print("  • Ask questions about PaaS configurations")
print("  • Get help with Kubernetes, Docker, Terraform")
print("  • Request example configurations")
print("  • Get best practices and guidelines")
print("\n🎮 Special commands:")
print("  • 'tools' - Show available agent tools")
print("  • 'config' - Show current configuration")
print("  • 'tokens' - Show token usage (if tracking enabled)")
print("  • 'exit' or 'quit' - End the session")
print("\n📚 Resources available:")
print("  • Documentation server: http://localhost:3000")
print("  • Local knowledge base (if loaded)")
print("  • Built-in PaaS expertise")
print("="*60)

print("\n🎯 Example questions to try:")
print("  • 'Create a simple web application deployment'")
print("  • 'What are the best practices for microservices?'")
print("  • 'Show me a Kubernetes deployment example'")
print("  • 'How do I set up load balancing?'")

print("\n⚡ Starting chat in 3 seconds...")
time.sleep(3)


In [ ]:
# Instructions to start the chat
print("🎯 TO START CHAT, RUN THIS COMMAND IN YOUR TERMINAL:")
print("="*60)
print("\npoetry run paas-ai agent chat --show-config")
print("\n💡 Or with your custom profile:")
print(f"poetry run paas-ai agent chat --config-profile {profile_name} --show-config")
print("\n" + "="*60)


## 🎉 Setup Complete!

Congratulations! You've successfully set up PaaS AI. Here's what we accomplished:

### ✅ What's Running:
1. **Dependencies Installed** - All Python packages via Poetry
2. **Custom Profile Created** - `notebook_demo` profile configured
3. **Documentation Server** - Running on http://localhost:3000
4. **Knowledge Base** - Ready for queries (loaded if you chose to)
5. **Chat Interface** - Ready to start

### 🚀 Next Steps:

1. **Open a terminal** and navigate to this project directory
2. **Run the chat command**:
   ```bash
   poetry run paas-ai agent chat --show-config
   ```
3. **Start asking questions** about PaaS configurations!

### 💡 Tips:
- Visit http://localhost:3000 to browse the documentation
- Use `poetry run paas-ai --help` to see all available commands
- Check `poetry run paas-ai config show` to see your current configuration
- Load more knowledge with `poetry run paas-ai rag index --help`

### 🛠️ Troubleshooting:
- If chat doesn't start, check your OpenAI API key in `.env`
- If documentation server isn't running, restart with `npm start` in `examples/paas/docs/`
- For configuration issues, try `poetry run paas-ai config init`

Happy coding with PaaS AI! 🤖✨


## 🧹 Cleanup (Optional)

When you're done experimenting, you can clean up resources:


In [ ]:
# Optional cleanup commands
print("🧹 Cleanup options:")
print("\n1. Stop documentation server:")
print("   - Press Ctrl+C in the terminal where npm start is running")
print("   - Or find and kill the process: pkill -f 'docusaurus start'")

print("\n2. Remove generated data (if you want to start fresh):")
print("   - rm -rf rag_data/  # Removes knowledge base")
print("   - rm -rf ~/.paas-ai/  # Removes conversation history")

print("\n3. Reset configuration:")
print("   - poetry run paas-ai config init  # Reinitialize config")

print("\n💡 You don't need to run cleanup unless you want to start over!")
